In [1]:
import wandb

import speechbrain
import torch
import torch.utils.data
# from torch.utils.data import DataLoader

import datasets
from datasets import load_dataset
import time
from rich.progress import Progress
from torchmetrics import WordErrorRate
import wandb
from speechbrain.pretrained import EncoderDecoderASR
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import json
datasets_names = ["ami","common_voice","earnings22","gigaspeech","librispeech","spgispeech","tedlium","voxpopuli"]
dataset_splits = ["clean","other"]


In [2]:
max_len = 0
storage = {}
for name in datasets_names:
    storage[name] = {}
    for split in dataset_splits:
        data_max = 0
        storage[name][split] = {}
        lens = []
        dataset = load_dataset("esb/diagnostic-dataset", name , split=split)
        for sample in dataset:
            lens.append(len(sample['norm_transcript'].split(' ')))
        max_len = max(max_len,max(lens))
        data_max = max(data_max,max(lens))
        storage[name][split]['token_lens'] = np.asarray(lens)
        storage[name][split]['total_tokens'] = sum(lens)
        storage[name][split]['total_samples'] = len(lens)
        storage[name][split]['max_len'] = data_max
        storage[name][split]['average'] = np.average(storage[name][split]['token_lens'])
        storage[name][split]['std'] = np.std(storage[name][split]['token_lens'])
        storage[name][split]['median'] = np.median(storage[name][split]['token_lens'])
        storage[name][split]['min'] = np.min(storage[name][split]['token_lens'])
        storage[name][split]['max'] = np.max(storage[name][split]['token_lens'])
# compute average of all datasets
storage['average'] = {}
for split in dataset_splits:
    max_sample_len = 0
    storage['average'][split] = {}
    lens = []
    for name in datasets_names:
        dataset = load_dataset("esb/diagnostic-dataset", name , split=split)
        for sample in dataset:
            lens.append(len(sample['norm_transcript'].split(' ')))
    max_len = max(max_len,max(lens))
    max_sample_len = max(max_sample_len,max(lens))
    storage["average"][split]['token_lens'] = np.asarray(lens)
    storage["average"][split]['total_tokens'] = sum(lens)
    storage["average"][split]['total_samples'] = len(lens)
    storage["average"][split]['max_len'] = max_sample_len
    storage['average'][split]['average'] = np.average(storage['average'][split]['token_lens'])
    storage['average'][split]['std'] = np.std(storage['average'][split]['token_lens'])
    storage['average'][split]['median'] = np.median(storage['average'][split]['token_lens'])
    storage['average'][split]['min'] = np.min(storage['average'][split]['token_lens'])
    storage['average'][split]['max'] = np.max(storage['average'][split]['token_lens'])


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating clean split: 0 examples [00:00, ? examples/s]

Generating other split: 0 examples [00:00, ? examples/s]

Dataset diagnostic-dataset downloaded and prepared to /Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/ami/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98. Subsequent calls will reuse this data.


Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/ami/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)


...

Generating clean split: 0 examples [00:00, ? examples/s]

Generating other split: 0 examples [00:00, ? examples/s]

Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/voxpopuli/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)


Dataset diagnostic-dataset downloaded and prepared to /Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/voxpopuli/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98. Subsequent calls will reuse this data.


Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/ami/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/common_voice/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/earnings22/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/gigaspeech/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/librispeech/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/

In [8]:
dataset[0]

{'audio': {'path': '/Users/micheleyin/.cache/huggingface/datasets/downloads/extracted/3d3cbc84ba0b9413c9fe0f34efcb2940101ba3eb263537b37d6ab5a45358b80e/other/20090204-0900-PLENARY-15-en_20090204-18:30:04_3.wav',
  'array': array([-0.11941528, -0.13668823, -0.13772583, ...,  0.00234985,
          0.0017395 ,  0.00234985], dtype=float32),
  'sampling_rate': 16000},
 'ortho_transcript': "Uh, couldn't we not do more, and could you perhaps be more specific, what has been done so far? What are your aims?",
 'norm_transcript': "uh couldn't we not do more and could you perhaps be more specific what has been done so far what are your aims",
 'id': '20090204-0900-PLENARY-15-en_20090204-18:30:04_3',
 'dataset': 'voxpopuli'}

In [48]:
medians = []
mins = []
maxs = []
total = 10
new_dataset = {}
for name in datasets_names:
    new_dataset[name] = {}
    for split in dataset_splits:
        new_dataset[name][split] = {}
        new_dataset[name][split]['median'] = []
        new_dataset[name][split]['min'] = []
        new_dataset[name][split]['max'] = []
        dataset = load_dataset("esb/diagnostic-dataset", name , split=split)
        sorted_dataset = sorted(dataset, key=lambda k: len(k['norm_transcript'].split(' ')))
        for sample in sorted_dataset:
            if len(new_dataset[name][split]['min']) < total:
                new_dataset[name][split]['min'].append(sample)
            else:
                break
        sorted_dataset = sorted(dataset, key=lambda k: len(k['norm_transcript'].split(' ')),reverse=True)
        for sample in sorted_dataset:
            if len(new_dataset[name][split]['max']) < total:
                new_dataset[name][split]['max'].append(sample)
            else:
                break
        sorted_dataset = sorted(dataset, key=lambda k: np.abs(len(k['norm_transcript'].split(' ')) -  np.average(storage[name][split]['token_lens'])))
        for sample in sorted_dataset:
            if len(new_dataset[name][split]['median']) < total:
                new_dataset[name][split]['median'].append(sample)
            else:
                break

Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/ami/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/ami/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/common_voice/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/common_voice/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache/huggingface/datasets/diagnostic-dataset/earnings22/1.0.0/1e12ea3493085c0ab38871cee98397cfae30b510e030b1b3722f2460a10d2e98)
Found cached dataset diagnostic-dataset (/Users/micheleyin/.cache

In [2]:
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-wav2vec2-commonvoice-en", savedir="pretrained_models/asr-wav2vec2-commonvoice-en")

Some weights of the model checkpoint at facebook/wav2vec2-large-lv60 were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.bias', 'project_q.weight', 'project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


'THE BIRCH CANOE SLID ON SMOOTH PLANKS'

In [49]:
wer = WordErrorRate()
for name in datasets_names:
    for split in dataset_splits:
        for k in new_dataset[name][split].keys():
            transcripts = []
            ground_truths = []
            tot_time = 0
            token_counts = 0
            wandb.init(
                # set the wandb project where this run will be logged
                project="ASR WER Test Local asr-wav2vec2-commonvoice-en",
                name = 'Local asr-wav2vec2-commonvoice-en'+"_"+name+"_"+split+"_"+k,
                # track hyperparameters and run metadata
                config={
                "architecture": 'Local asr-wav2vec2-commonvoice-en',
                "dataset": name,
                "split": split,
                'partition' : k
                }
            )
            for sample in new_dataset[name][split][k]:
                t = torch.tensor(sample['audio']['array'])
                t = t.reshape(1,-1)
                lens = torch.ones((1))
                timer = time.time()
                trascript,_= asr_model.transcribe_batch(t,lens)
                timer = time.time() - timer
                tot_time += timer
                transcripts.append(trascript[0].lower())
                ground_truths.append(sample['norm_transcript'])
                token_counts += len(sample['norm_transcript'].split(' '))
            average_time = tot_time/len(new_dataset[name][split])
            wer_score = wer(ground_truths, transcripts)
            wandb.log({"WER": wer_score})
            wandb.log({"Tokens per second": token_counts/tot_time})
            wandb.log({"Average time per sample": average_time})
            wandb.log({"Total time": tot_time})
            wandb.log({"Total samples": len(dataset)})
            wandb.log({"Total tokens": token_counts})
            wandb.finish()


wandb: Currently logged in as: bigemperor26 (micheleyin). Use `wandb login --relogin` to force relogin


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,1.35698
Tokens per second,19.65147
Total samples,210
Total time,4.07094
Total tokens,80


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,0.58526
Tokens per second,5.69548
Total samples,210
Total time,1.75578
Total tokens,10


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,8.63912
Tokens per second,13.08003
Total samples,210
Total time,25.91737
Total tokens,339


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,1.04363
Tokens per second,25.55176
Total samples,210
Total time,3.1309
Total tokens,80


...

Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,0.94794
Tokens per second,15.82385
Total samples,210
Total time,2.84381
Total tokens,45


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,22.57453
Tokens per second,9.21392
Total samples,210
Total time,67.72359
Total tokens,624


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,5.77252
Tokens per second,14.08974
Total samples,210
Total time,17.31756
Total tokens,244


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,0.90472
Tokens per second,16.21134
Total samples,210
Total time,2.71415
Total tokens,44


Average time per sample,▁
Tokens per second,▁
Total samples,▁
Total time,▁
Total tokens,▁
WER,▁
Average time per sample,22.45279
Tokens per second,10.74848
Total samples,210
Total time,67.35838
Total tokens,724
